In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import tensorboard
from sklearn.model_selection import train_test_split

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Utils')
sys.path.append('/content/drive/MyDrive/Utils/DL/tensorflowUtils')

In [ ]:
import ntfy

In [ ]:
import info
info.get_version_info()

AttributeError: ignored

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/Digit-Recognizer/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Digit-Recognizer/test.csv")

train_df.shape, test_df.shape

((42000, 785), (28000, 784))

In [ ]:
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# convert to 28x28 matrix, withouth label col

# train data withouth label 
train_images = train_df.iloc[:,1:].values
# reshape to 28x28 -> -1 for get all the data in shape (x,), then 28x28 (size of matrix), and 1 for color chanel (if RGB 1 will be 3)
train_images = train_images.reshape(-1,28,28,1)
# Normalization (pixels goes from 0 to 255 range, so we divide it by 255 and then will be between 0 and 1)
train_images = train_images/255.0
train_images[0]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_images,
                                                    train_df['label'],
                                                    train_size=0.2,
                                                    random_state=42)

In [ ]:
X_train.shape, y_train.shape

((8400, 28, 28, 1), (8400, 1))

In [ ]:
tf.random.set_seed(42)

INPUT_SHAPE = [None, 28, 28, 1]

# Setup the layers
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=100, activation='relu'),
    tf.keras.layers.Dense(units=100, activation='relu'),
    tf.keras.layers.Dense(10, activation='sigmoid')
])

# Compile model
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.build(INPUT_SHAPE)

In [ ]:
model.summary()

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_11 (Flatten)        (None, 784)               0         
                                                                 
 dense_71 (Dense)            (None, 100)               78500     
                                                                 
 dense_72 (Dense)            (None, 100)               10100     
                                                                 
 dense_73 (Dense)            (None, 10)                1010      
                                                                 
Total params: 89,610
Trainable params: 89,610
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train, epochs=30)

Epoch 1/30
263/263 [==============================] - 3s 4ms/step - loss: 0.5929 - accuracy: 0.8392
Epoch 2/30
263/263 [==============================] - 1s 4ms/step - loss: 0.2345 - accuracy: 0.9317
Epoch 3/30
263/263 [==============================] - 1s 3ms/step - loss: 0.1627 - accuracy: 0.9529
Epoch 4/30
263/263 [==============================] - 1s 3ms/step - loss: 0.1168 - accuracy: 0.9658
Epoch 5/30
263/263 [==============================] - 1s 3ms/step - loss: 0.0858 - accuracy: 0.9750
Epoch 6/30
263/263 [==============================] - 1s 3ms/step - loss: 0.0663 - accuracy: 0.9805
Epoch 7/30
263/263 [==============================] - 1s 3ms/step - loss: 0.0467 - accuracy: 0.9861
Epoch 8/30
263/263 [==============================] - 1s 3ms/step - loss: 0.0368 - accuracy: 0.9900
Epoch 9/30
263/263 [==============================] - 1s 3ms/step - loss: 0.0318 - accuracy: 0.9908
Epoch 10/30
263/263 [==============================] - 1s 3ms/step - loss: 0.0200 - accuracy: 0.9949

In [ ]:
model.evaluate(X_test, y_test)

1050/1050 [==============================] - 4s 3ms/step - loss: 0.2433 - accuracy: 0.9572


[0.2433418184518814, 0.9572321176528931]

In [ ]:
model.save("/content/drive/MyDrive/Digit-Recognizer/model")

In [ ]:
model = tf.keras.models.load_model("/content/drive/MyDrive/Digit-Recognizer/model",
                                     custom_objects={'KerasLayer':hub.KerasLayer})

In [ ]:
test_images = test_df.values.reshape(-1,28,28,1)/255.0
preds = np.argmax(model.predict(test_images), axis=1)

875/875 [==============================] - 2s 2ms/step


In [ ]:
preds

array([2, 0, 9, ..., 3, 9, 2])

In [ ]:
submission_df = pd.DataFrame({'ImageId':test_df.index+1, 'Label':preds})
submission_df.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [ ]:
submission_df.to_csv('NN-simple.csv', index=False)